# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

In [60]:
import numpy as np
import torch
from torchvision import datasets,transforms
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
###LOADING THE DATA

In [15]:
# Transform the image to a tensor and normalize it
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Load the train and test data by using the transform
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

In [16]:
###CREATING TEACHERS DATASETS

In [17]:
num_teachers = 100
batch_size = 32

In [18]:
def get_data_loaders(train_data, num_teachers):
    teacher_loaders = []
    data_size = len(train_data)//num_teachers
    for i in range(data_size):
        indices = list(range(i*data_size, (i+1)*data_size))
        subset_data = Subset(train_data,indices)
        loader = torch.utils.data.DataLoader(subset_data, batch_size = batch_size)
        teacher_loaders.append(loader)
    
    return teacher_loaders
teacher_loaders = get_data_loaders(train_data,num_teachers)

In [19]:
### CREATING STUDENTS DATASET

In [24]:
student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000,10000)))
                            
student_train_loader = torch.utils.data.DataLoader(student_train_data,batch_size = batch_size)
student_test_loader = torch.utils.data.DataLoader(student_test_data,batch_size = batch_size)

In [25]:
###BUILDING A CLASSIFIER

In [50]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20,kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
        

In [51]:
###TRAINING PIPELINE

In [83]:
def train(model,trainloader,criterion,optimizer,epochs = 10):
    running_loss = 0
    for e in range(epochs):
        model.train()
    
        for images,labels in trainloader:
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output,labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            

In [84]:
###PREDICTION PIPELINE

In [85]:
def predict(model,dataloader):
    outputs = torch.zeros(0,dtype = torch.long)
    model.eval()
    
    for images,labels in dataloader:
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output),dim = 1)
        outputs = torch.cat((outputs,ps))
        
    return outputs

In [86]:
def train_models(num_teachers):
    models = []
    for i in range(num_teachers):
        model  = Classifier()
        criterion  = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.003)
        train(model,teacher_loaders[i],criterion,optimizer)
        models.append(model)
        
    return models
models = train_models(num_teachers)

/home/jpmc/anaconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [87]:
###CREATING STUDENTS SYNTHETIC LABELS

In [88]:
epsilon = 0.2

def aggregated_teacher(models,dataloader,epsilon):
    preds = torch.torch.zeros(len(models),9000, dtype = torch.long)
    for i,model in enumerate(models):
        results = predict(model,dataloader)
        preds[i] = results
    
    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds):
        label_counts = np.bincount(image_preds, minlength = 10)
        beta = 1/epsilon
        
        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0,beta,1)
        
        new_label = np.argmax(label_counts)
        labels = np.append(labels,new_label)
    
    return preds.numpy(),labels

teacher_models = models
preds,student_labels = aggregated_teacher(teacher_models,student_train_loader,epsilon)



/home/jpmc/anaconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [89]:
###TRAINING STUDENT MODEL

In [93]:
def student_loader(student_train_loader,labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data,torch.from_numpy(labels[i*len(data):(i+1)*len(data)])
student_model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr = 0.003)
epochs = 10
steps = 0
running_loss = 0
for e in range(epochs):
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels)
    for images,labels in train_loader:
        steps +=1
        
        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
        
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            
            with torch.no_grad():
                for images,labels in student_test_loader:
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps,labels).item()
                    
                    ps = torch.exp(log_ps)
                    top_p , top_class = ps.topk(1,dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                    student_model.train()
                     print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0
        
        
        
        

/home/jpmc/anaconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.049..  Test Accuracy: 0.015
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.100..  Test Accuracy: 0.028
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.135..  Test Accuracy: 0.047
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.169..  Test Accuracy: 0.067
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.201..  Test Accuracy: 0.089
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.241..  Test Accuracy: 0.104
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.289..  Test Accuracy: 0.120
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.345..  Test Accuracy: 0.133
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.377..  Test Accuracy: 0.155
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.410..  Test Accuracy: 0.177
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.446..  Test Accuracy: 0.195
Epoch: 1/10..  Training Loss: 0.340..  Test Loss: 0.476..  Test Accuracy: 0.218
Epoch: 1/10..  Training Loss: 0.340..  T

Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.278..  Test Accuracy: 0.474
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.292..  Test Accuracy: 0.500
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.320..  Test Accuracy: 0.522
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.349..  Test Accuracy: 0.543
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.370..  Test Accuracy: 0.567
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.420..  Test Accuracy: 0.587
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.476..  Test Accuracy: 0.604
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.541..  Test Accuracy: 0.619
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.563..  Test Accuracy: 0.643
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.582..  Test Accuracy: 0.667
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.616..  Test Accuracy: 0.688
Epoch: 1/10..  Training Loss: 0.081..  Test Loss: 0.648..  Test Accuracy: 0.709
Epoch: 1/10..  Training Loss: 0.081..  T

Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.015..  Test Accuracy: 0.027
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.040..  Test Accuracy: 0.051
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.045..  Test Accuracy: 0.081
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.059..  Test Accuracy: 0.109
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.068..  Test Accuracy: 0.139
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.080..  Test Accuracy: 0.166
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.101..  Test Accuracy: 0.191
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.116..  Test Accuracy: 0.218
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.127..  Test Accuracy: 0.245
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.133..  Test Accuracy: 0.275
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.137..  Test Accuracy: 0.305
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.141..  Test Accuracy: 0.336
Epoch: 2/10..  Training Loss: 0.079..  T

Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.173..  Test Accuracy: 0.507
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.176..  Test Accuracy: 0.538
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.209..  Test Accuracy: 0.561
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.233..  Test Accuracy: 0.587
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.246..  Test Accuracy: 0.613
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.267..  Test Accuracy: 0.635
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.280..  Test Accuracy: 0.661
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.336..  Test Accuracy: 0.683
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.352..  Test Accuracy: 0.709
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.368..  Test Accuracy: 0.734
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.392..  Test Accuracy: 0.757
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.415..  Test Accuracy: 0.781
Epoch: 2/10..  Training Loss: 0.053..  T

...

Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.030..  Test Accuracy: 0.026
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.045..  Test Accuracy: 0.054
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.052..  Test Accuracy: 0.084
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.062..  Test Accuracy: 0.111
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.072..  Test Accuracy: 0.140
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.083..  Test Accuracy: 0.166
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.097..  Test Accuracy: 0.194
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.105..  Test Accuracy: 0.224
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.114..  Test Accuracy: 0.254
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.120..  Test Accuracy: 0.282
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.123..  Test Accuracy: 0.313
Epoch: 8/10..  Training Loss: 0.043..  Test Loss: 0.125..  Test Accuracy: 0.344
Epoch: 8/10..  Training Loss: 0.043..  T

Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.129..  Test Accuracy: 0.562
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.146..  Test Accuracy: 0.588
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.169..  Test Accuracy: 0.613
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.185..  Test Accuracy: 0.640
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.215..  Test Accuracy: 0.664
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.237..  Test Accuracy: 0.688
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.262..  Test Accuracy: 0.713
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.265..  Test Accuracy: 0.744
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.282..  Test Accuracy: 0.769
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.304..  Test Accuracy: 0.796
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.318..  Test Accuracy: 0.822
Epoch: 9/10..  Training Loss: 0.072..  Test Loss: 0.329..  Test Accuracy: 0.852
Epoch: 9/10..  Training Loss: 0.072..  T

Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.031..  Test Accuracy: 0.026
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.050..  Test Accuracy: 0.053
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.057..  Test Accuracy: 0.080
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.074..  Test Accuracy: 0.108
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.085..  Test Accuracy: 0.138
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.092..  Test Accuracy: 0.167
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.096..  Test Accuracy: 0.197
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.102..  Test Accuracy: 0.228
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.111..  Test Accuracy: 0.257
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.115..  Test Accuracy: 0.287
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.118..  Test Accuracy: 0.317
Epoch: 9/10..  Training Loss: 0.032..  Test Loss: 0.120..  Test Accuracy: 0.349
Epoch: 9/10..  Training Loss: 0.032..  T

Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.125..  Test Accuracy: 0.560
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.134..  Test Accuracy: 0.588
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.157..  Test Accuracy: 0.614
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.177..  Test Accuracy: 0.637
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.204..  Test Accuracy: 0.660
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.226..  Test Accuracy: 0.684
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.252..  Test Accuracy: 0.708
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.257..  Test Accuracy: 0.737
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.272..  Test Accuracy: 0.767
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.291..  Test Accuracy: 0.793
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.306..  Test Accuracy: 0.816
Epoch: 10/10..  Training Loss: 0.048..  Test Loss: 0.313..  Test Accuracy: 0.846
Epoch: 10/10..  Training Los

Epoch: 10/10..  Training Loss: 0.037..  Test Loss: 0.279..  Test Accuracy: 0.764
Epoch: 10/10..  Training Loss: 0.037..  Test Loss: 0.319..  Test Accuracy: 0.786
Epoch: 10/10..  Training Loss: 0.037..  Test Loss: 0.331..  Test Accuracy: 0.812
Epoch: 10/10..  Training Loss: 0.037..  Test Loss: 0.336..  Test Accuracy: 0.842
Epoch: 10/10..  Training Loss: 0.037..  Test Loss: 0.349..  Test Accuracy: 0.870
Epoch: 10/10..  Training Loss: 0.037..  Test Loss: 0.350..  Test Accuracy: 0.901


In [94]:
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=student_labels, noise_eps=epsilon, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 1451.5129254649705
Data Dependent Epsilon: 16.665037101642003
